# Model Garden | Content Classification

Google has 2 NLP versions:

- *google/language_v1-classify_text_v2*	
- *google/language_v1-classify_text_v1*	

## Function that Calls the  Prebuilt Model (NLP)

In [1]:
from google.cloud import language_v1


def sample_classify_text(text_content):
    """
    Classifying Content in a String

    Args:
      text_content The text content to analyze.
    """

    client = language_v1.LanguageServiceClient()

    # text_content = "That actor on TV makes movies in Hollywood and also stars in a variety of popular new TV shows."

    # Available types: PLAIN_TEXT, HTML
    type_ = language_v1.Document.Type.PLAIN_TEXT

    language = "en"
    document = {"content": text_content, "type_": type_, "language": language}

    content_categories_version = (
        language_v1.ClassificationModelOptions.V2Model.ContentCategoriesVersion.V2
    )
    response = client.classify_text(
        request={
            "document": document,
            "classification_model_options": {
                "v2_model": {"content_categories_version": content_categories_version}
            },
        }
    )
    # Loop through classified categories returned from the API
    for category in response.categories:

        print("Category name: {}".format(category.name))
        print("Confidence: {}".format(category.confidence))

## Loading Dataset from Google Cloud BigQuery (gdelt-bq.covid19.onlinenews)

2 steps:

1. Load data from BigQuery
2. Use the above function to classify

In [2]:
from google.cloud import bigquery

client = bigquery.Client(project="vtxdemos")
sql = "SELECT * FROM gdelt-bq.covid19.onlinenews LIMIT 5"
dataset = client.query(sql).to_dataframe()

for n, content in enumerate(dataset["Context"].tolist()):
    print("################################ --- START -- ################################")
    print(f"Prompt: {content}")
    sample_classify_text(content)
    print("################################ --- START -- ################################ \n\n\n")

################################ --- START -- ################################
Prompt: This article is interesting for a number of reasons. The first is that the Department of Disease Control does not know what exactly is causing the worst wave of viral infection in China's history. The Communist Party of China has stopped all testing for Covid-19 and has released no information regarding sequencing or variant characterisation of the pathogen infecting millions of their citizens.
Category name: /Health/Health Conditions/Infectious Diseases/Covid-19
Confidence: 0.875003457069397
Category name: /Health/Public Health/Other
Confidence: 0.5517397522926331
Category name: /News/Health News
Confidence: 0.5002300143241882
Category name: /Health/Medical Literature & Resources/Other
Confidence: 0.21246251463890076
Category name: /Health/Health Conditions/Infectious Diseases/Other
Confidence: 0.10606330633163452
################################ --- START -- ################################ 



###